In [93]:
import numpy as np
import pandas as pd
import matplotlib as plt
import os
from bs4 import BeautifulSoup
import re

import nltk.corpus
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

from textblob import TextBlob
import html2text

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\swani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [94]:
os.chdir(r"C:\Users\swani\Python Projects\Case Bucket\Data")

In [95]:
data_all = pd.read_excel('IT-706405.xlsx')
length = int(len(data_all))
length

C:\Users\swani\AppData\Local\Temp/ipykernel_30184/2741710585.py:1: FutureWarning: Inferring datetime64[ns, UTC] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  data_all = pd.read_excel('IT-706405.xlsx')


3736

In [96]:
for i in data_all.columns:
    print(i)

ACCESS_RIGHTS__C
ADDITIONAL_COMMENTS__C
ANSWER__C
ARTICLE_NUMBER__C
AUTHOR_REGION__C
AUTHOR_S__C
AUTHORITATIVE_PAGE__C
AUTHORSID__C
BLACKLIST_ACCOUNTID__C
BLACKLIST_CONTACT_ID__C
BLACKLIST_DOMAIN__C
BLACK_LIST__C
BLACKLIST__C
CCR__C
CDN_PLATFORM__C
COS_CONTENT_URL__C
COS_OPEN__C
COS_ORIGINAL_DATE__C
COS_PUBLIC_DATE__C
COS_SESSION_ID__C
CADENCECREATIONDATE__C
CADENCEFOLDERPATH__C
CADENCEMODIFYDATE__C
CADENCE_PLATFORM__C
CHANNEL__C
CLONED_ATTACHMENT_ID__C
CLONED_FEED_ID__C
CLONED_NOTE_ID__C
CLOSE_EARS__C
CLOSED_EAR_FLAG_DATE__C
CODE__C
CONTENT_CREATION_DATE__C
CONTENT_SOURCE__C
CREATEDBYID
CREATEDDATE
CREATIONDATEWP__C
CRITICAL__C
CROWD_SOURCED_SEARCH_KEYWORDS__C
CURRENCYISOCODE
DATE_SPECIFIC_PUBLISHED_DATE__C
DATE_OF_NOTIFICATION__C
DATE_OF_REVIEW__C
DATE_OF_NOTIFICATION_SENT__C
DEFINITION__C
DELETED_ATTACHMENT__C
DELIVERY__C
DESCRIPTION__C
DESIGN_TOPIC__C
DOCUMENT_AUTHOR_NAME__C
DOCUMENT_CATEGORY__C
DOCUMENT_TYPES__C
DRAFT_ANSWER__C
DRAFT_CODE__C
DRAFT_DEFINITION__C
DRAFT_ERROR_MESSAGE

In [97]:
data_required = data_all[['ANSWER__C','ARTICLE_NUMBER__C','CONTENT_SOURCE__C','DEFINITION__C','DESCRIPTION__C','DOCUMENT_CATEGORY__C','DOCUMENT_TYPES__C','HOW_TO__C','NAME','PROBLEM__C','SOLUTION__C','SUMMARY__C','TITLE__C']]

In [98]:
data_required.head()

,ANSWER__C,ARTICLE_NUMBER__C,CONTENT_SOURCE__C,DEFINITION__C,DESCRIPTION__C,DOCUMENT_CATEGORY__C,DOCUMENT_TYPES__C,HOW_TO__C,NAME,PROBLEM__C,SOLUTION__C,SUMMARY__C,TITLE__C
0,NaN,538672,NaN,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,NaN,NaN,NaN,NaN,20467935.0,NaN,<p>This is correct behavior as per the LRM as ...,User-defined resolution functions can be writt...,Multiple driver resolution using user-defined ...
1,NaN,538690,NaN,NaN,NaN,NaN,NaN,NaN,20467953.0,<p>User is attempting to set an enumerated typ...,<p>It is not possible to directly pass an enum...,User is unable to set an enumerated type varia...,Passing enumeration type variable using uvm co...
2,NaN,538696,NaN,NaN,NaN,NaN,Application Notes;Rapid Adoption Kits;Videos;W...,NaN,20467959.0,NaN,NaN,Xcelium Multi-Core is a 3rd generation simulat...,One-Stop Knowledge Resource for Xcelium Multi-...
3,NaN,538719,NaN,NaN,NaN,NaN,Application Notes;Rapid Adoption Kits;What's New,NaN,20467982.0,NaN,NaN,This is a one-stop resource for Xcelium Integr...,One-Stop Knowledge Resource for Xcelium Integr...
4,NaN,538720,NaN,NaN,NaN,NaN,Static Pages,NaN,20467983.0,NaN,NaN,This page collects information related to Func...,Functional Safety Overview and Content References


### Combining the Answer and Solution data fileds

In [99]:
final_list = []
for i in range(len(data_required)):
    if(pd.isnull(data_required['PROBLEM__C'][i])):
        if(pd.isnull(data_required['HOW_TO__C'][i])):
            if(pd.isnull(data_required['DEFINITION__C'][i])):
                if(pd.isnull(data_required['DESCRIPTION__C'][i])):
                    if(pd.isnull(data_required['SUMMARY__C'][i])):
                        final_list.append('None')
                    else:
                        final_list.append(data_required['SUMMARY__C'][i])
                else:
                    final_list.append(data_required['DESCRIPTION__C'][i])
            else:
                final_list.append(data_required['DEFINITION__C'][i])
        else:
            final_list.append(data_required['HOW_TO__C'][i])
    else:
        final_list.append(data_required['PROBLEM__C'][i])

data_required['Problem'] = final_list

C:\Users\swani\AppData\Local\Temp/ipykernel_30184/1996066343.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_required['Problem'] = final_list


In [100]:
data_required.head(1)

,ANSWER__C,ARTICLE_NUMBER__C,CONTENT_SOURCE__C,DEFINITION__C,DESCRIPTION__C,DOCUMENT_CATEGORY__C,DOCUMENT_TYPES__C,HOW_TO__C,NAME,PROBLEM__C,SOLUTION__C,SUMMARY__C,TITLE__C,Problem
0,NaN,538672,NaN,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,NaN,NaN,NaN,NaN,20467935.0,NaN,<p>This is correct behavior as per the LRM as ...,User-defined resolution functions can be writt...,Multiple driver resolution using user-defined ...,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...


In [101]:
data_required = data_required.drop(data_required.columns[[0,2,3,4,5,7,9,10,11]], axis=1)
data_required.head(1)

,ARTICLE_NUMBER__C,DOCUMENT_TYPES__C,NAME,TITLE__C,Problem
0,538672,NaN,20467935.0,Multiple driver resolution using user-defined ...,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...


In [102]:
data_required['Problem'] = data_required['Problem'].astype(str)

In [103]:
a = []
for i in data_required['Problem']:
    j = BeautifulSoup(i).get_text()
    a.append(j)

data_required['Problem']=a
data_required.head()

C:\Users\swani\Anaconda3\lib\site-packages\bs4\__init__.py:337: MarkupResemblesLocatorWarning: "." looks like a directory name, not markup. You may want to open a file found in this directory and pass the filehandle into Beautiful Soup.
  warnings.warn(


,ARTICLE_NUMBER__C,DOCUMENT_TYPES__C,NAME,TITLE__C,Problem
0,538672,NaN,20467935.0,Multiple driver resolution using user-defined ...,For VHDL:\n You get the CSGMSS error for da...
1,538690,NaN,20467953.0,Passing enumeration type variable using uvm co...,User is attempting to set an enumerated type v...
2,538696,Application Notes;Rapid Adoption Kits;Videos;W...,20467959.0,One-Stop Knowledge Resource for Xcelium Multi-...,Xcelium Multi-Core is a 3rd generation simulat...
3,538719,Application Notes;Rapid Adoption Kits;What's New,20467982.0,One-Stop Knowledge Resource for Xcelium Integr...,This is a one-stop resource for Xcelium Integr...
4,538720,Static Pages,20467983.0,Functional Safety Overview and Content References,This page collects information related to Func...


In [104]:
data_required['Problem'] = data_required['Problem'].str.replace(r'\n', '')

C:\Users\swani\AppData\Local\Temp/ipykernel_30184/3156732733.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data_required['Problem'] = data_required['Problem'].str.replace(r'\n', '')


In [105]:
data_required.head()

,ARTICLE_NUMBER__C,DOCUMENT_TYPES__C,NAME,TITLE__C,Problem
0,538672,NaN,20467935.0,Multiple driver resolution using user-defined ...,For VHDL: You get the CSGMSS error for data...
1,538690,NaN,20467953.0,Passing enumeration type variable using uvm co...,User is attempting to set an enumerated type v...
2,538696,Application Notes;Rapid Adoption Kits;Videos;W...,20467959.0,One-Stop Knowledge Resource for Xcelium Multi-...,Xcelium Multi-Core is a 3rd generation simulat...
3,538719,Application Notes;Rapid Adoption Kits;What's New,20467982.0,One-Stop Knowledge Resource for Xcelium Integr...,This is a one-stop resource for Xcelium Integr...
4,538720,Static Pages,20467983.0,Functional Safety Overview and Content References,This page collects information related to Func...


### Remove Stopwords

In [106]:
stop_words = stopwords.words('english')
data_required['Problem'] = data_required['Problem'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

### Remove Keywords flagged by IT Stopwords

In [107]:
df = pd.read_excel('ALl-keywords-2021.xlsx')
exceptions = df["Keywords Flagged by IT"].tolist()
data_required['Problem'] = data_required['Problem'].apply(lambda x: ' '.join([word for word in x.split() if word not in (exceptions)]))

In [88]:
exceptions

['simulation',
 'ref',
 'taken ownership',
 'understanding',
 'support database',
 'cadence',
 'intended recipient',
 'Reference',
 'attachments',
 'ArticleAttachmentPortal',
 'compilation',
 'xcelium',
 'elaboration',
 'open attachments',
 'design challenges',
 'simulations',
 'Venkatesh Lingaiah',
 'unexpected condition',
 'Sumit Sharma',
 'customer',
 'webex session',
 'follow-up query',
 'dissemination',
 'workaround',
 'command',
 'Cadence Support',
 'customer support',
 'Design Systems',
 'customers',
 'Product Feature',
 'webex',
 'Rajat Gupta',
 'legally privileged',
 'email originated',
 'Cadence Design Systems',
 'extended help',
 'Western Digital',
 'ArticleContent',
 'warnings',
 'Hiren Shah',
 'Ashutosh Gupta',
 'exit',
 'sender',
 'Xcelium Product Feature',
 'Atul Kumar',
 'Vivek Raheja',
 'articles',
 'unauthorized review',
 'use of the individual',
 'root cause',
 'Regards',
 'Services Agreement',
 'privacy laws',
 'availability',
 'Xcelium Product',
 'salesforce',
 'Pr

In [111]:
len(data_required)

3736

### Lemmatization

In [110]:
lmtzr = WordNetLemmatizer()
lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in data_required['Problem']]
data_required['Problem'] = lemmatized
data_required['Problem'] = [' '.join(map(str, l)) for l in data_required['Problem']]

data.head()

,Name,Answer__c,CreatedDate,Article_Number__c,Author_Region__c,Cadence_Platform__c,Channel__c,Critical__c,Definition__c,External_Family__c,...,Problem__c,Product_Feature2__r.Name,Product_Feature_Lvl_1__r.Name,Product_Feature_Lvl_2__r.Name,Product_Feature_Version__r.Name,Product_Feature__r.Name,Product__r.Name,Published_Date__c,Saved_Solution__c,Solution__c
0,20467935,NaN,2017-06-13T06:27:28.000+0000,538672.0,GCS,Functional Verification,Customer,0.0,<p><strong>For VHDL:</strong></p>\n\n<p> Yo...,Xcelium,...,NaN,Single Core,Multi-Language Sim,LRM Compliance,None,Single Core,Xcelium,2017-07-14T09:00:55.000+0000,<p>This is correct behavior as per the LRM as ...,<p>This is correct behavior as per the LRM as ...
1,20467953,NaN,2017-06-13T21:36:58.000+0000,538690.0,NACS,Functional Verification,Customer,0.0,NaN,Xcelium,...,<p>User is attempting to set an enumerated typ...,Single Core,UVM,UVM SV,None,Single Core,Xcelium,2017-06-13T22:11:03.000+0000,<p>It is not possible to directly pass an enum...,<p>It is not possible to directly pass an enum...
2,20467959,NaN,2017-06-14T13:39:23.000+0000,538696.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Multi Core,Documentation,NaN,20.03,Multi Core,Xcelium,2017-06-14T13:39:23.000+0000,NaN,NaN
3,20467982,NaN,2017-06-15T11:18:38.000+0000,538719.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Single Core,Comprehensive Coverage,Simulation/Recording,20.03,Single Core,Xcelium,2017-06-15T11:18:38.000+0000,NaN,NaN
4,20467983,NaN,2017-06-15T11:49:36.000+0000,538720.0,GCS,Functional Verification,Customer,0.0,NaN,Xcelium,...,NaN,Functional Safety,Documentation,NaN,17.04,Functional Safety,Xcelium,NaN,NaN,NaN


In [112]:
datatoexcel = pd.ExcelWriter('N-Grams KB Articles Processes Data.xlsx')
  
# write DataFrame to excel
data_required.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()